In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2 as cv
import os

os.chdir('/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data')
cv.__version__

'4.1.2'

In [ ]:
!ls

download.zip  PET.zip  PE.zip  PP.zip  PS.zip  Test  Train


In [ ]:
def cvColor():
  # R, G, B
  BLACK = (0, 0, 0)
  WHITE = (255, 255, 255)
  GRAY = (125, 125, 125)
  RED = (255, 0, 0)
  GREEN = (0, 255, 0)
  BLUE = (0, 0, 255)
  CYAN = (0, 255, 255)
  MAGENTA = (255, 0, 255)
  YELLOW = (255, 255, 0)
  PINK = (238, 130, 238)
  ORANGE = (255, 165, 0)
  MINT = (60, 179, 113)
  LAVENDER = (106, 90, 205)
  IVORY = (240, 240, 240)
  SALMON = (240, 150, 120)
  colors = {"RED":RED, "GREEN":GREEN, "BLUE":BLUE, \
          "MAGENTA":MAGENTA, "CYAN":CYAN, "YELLOW":YELLOW, \
          "WHITE":WHITE, "GRAY":GRAY, "BLACK":BLACK, \
          "PINK":PINK, "ORANGE":ORANGE, "MINT":MINT, \
          "LAVENDER":LAVENDER, "IVORY":IVORY, "SALMON":SALMON}
  return colors

colors = cvColor()

In [ ]:
from PIL import Image
import random
from glob import glob
from matplotlib import patches,  lines
from matplotlib.patches import Polygon
import json
import skimage

PATH = 'Train/annotation'
# 4개의 directory에서 하나씩 뽑아서 샘플을 가지고 segmentation 확인하기
dir_list = ['PE','PET','PP','PS']

# _, ax = plt.subplots(4, figsize = (22, 17))

color = {k:i for k,i in enumerate(colors.keys())}

for i in dir_list:
    
    plt.figure(figsize= (16,16))

  
    # 각 directory의 annottation에서 json파일을 가져온다.
    data = glob(PATH + '/' + i + '/' + '*.json')
  
    # json중 하나만 뽑아서 가져온다.
    json_data = random.choice(data)
  
    # json 파일을 열어서 annot에 저장한다.
    with open(json_data, 'r') as f:
        annot = json.loads(f.read())

    # image의 이름을 가져온다.
    image_name = annot['images'][0]['file_name']

    # image를 연다.
    image = Image.open('train' + '/' + i + '/' + image_name)

    # 이미지를 넘파이 형태로 저장한다.
    image = np.array(image)
  
    width, height, channel = image.shape[0], image.shape[1], image.shape[2]

    masked_image = np.zeros((width, height), dtype = np.uint8)

    # object의 구조상 몇 개의 사물이 관측될지 모르기 때문에 annot의 annotations의 개수를 가져온다.
    for j in range(len(annot['annotations'])):

        # annotation에서 각 번호에 맞는 segmentation을 가져온다.
        segmentation = annot['annotations'][j]['segmentation'][0]

        # annotation에서 각 번호에 맞는 bbox를 가져온다.
        bbox = annot['annotations'][j]['bbox']

        label = annot['annotations'][j]['category_id']

        xmin, ymin, width, height = bbox[0],bbox[1],bbox[2],bbox[3]
        cv.rectangle(image, (xmin, ymin), (xmin + width, ymin + height),colors[color[j]],4)

        # segmentation
        all_points_x = []
        all_points_y = []

        for i in range(len(segmentation)):
            if i%2 == 0:
                all_points_x.append(segmentation[i])
            else:
                all_points_y.append(segmentation[i])

        polygon_xy = [(x,y) for (x,y) in zip(all_points_x, all_points_y)]

        cv.fillPoly(image, np.uint([polygon_xy]), colors[color[j]])
    
    plt.imshow(image)
    plt.show()

Output hidden; open in https://colab.research.google.com to view.